In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from seaborn import set_style
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report,balanced_accuracy_score, ConfusionMatrixDisplay, make_scorer, roc_auc_score
from sklearn.svm import SVC

In [43]:
PD_merged_all = pd.read_csv("C:/Users/dcsor/Desktop/Parkinson's Project/merged_all.csv")
PD_merged_all = PD_merged_all.dropna()

NHY Target

In [55]:
Y_Target = PD_merged_all.NHY

def combine_class_three(x):
    if x >= 3:
       return 2
    elif 1 <= x < 3:
        return 1
    else:
        return 0 
def combine_class(x):
    if x >= 1:
        return 1
    else:
        return 0
    
Y_target = Y_Target.apply(combine_class_three)
Y_target.value_counts(normalize=True)

NHY
1    0.438185
0    0.427230
2    0.134585
Name: proportion, dtype: float64

Baseline Data 



In [56]:
Baseline_features = PD_merged_all[["GENDER", "AGE", "EDUC_YRS"]]
pd.get_dummies(Baseline_features['GENDER'],dtype=int)
Baseline_features['Male'] = pd.get_dummies(Baseline_features['GENDER'],dtype=int)['Male']
X_Baseline = Baseline_features.drop(columns=['GENDER'])

C:\Users\dcsor\AppData\Local\Temp\ipykernel_30668\1279131828.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Baseline_features['Male'] = pd.get_dummies(Baseline_features['GENDER'],dtype=int)['Male']


In [57]:
#Split data 
X_train, X_test , y_train , y_test = train_test_split(X_Baseline, Y_target,
                                            shuffle=True,
                                            random_state=42,
                                            test_size=.2,
                                            stratify=Y_target)

#Make validation set with 15% of the
## training data in the validation set
X_train_train, X_val, y_train_train, y_val = train_test_split(
    X_train, y_train, shuffle=True, random_state=42, test_size=0.15, stratify=y_train
)

kfold = StratifiedKFold(n_splits=5,
                           shuffle=True,
                           random_state=42)
kfold.get_n_splits(X_train,y_train)


SVC_pipe = Pipeline([('scale', StandardScaler()), ('SVC', SVC(kernel = 'rbf', C=10, probability= True))])

for i, (train_index, test_index) in enumerate(kfold.split(X_train, y_train)):
    ## get the kfold training data
    X_train_train = X_train.iloc[train_index, :]
    y_train_train = y_train.iloc[train_index]

    ## get the holdout data
    X_holdout = X_train.iloc[test_index, :]
    y_holdout = y_train.iloc[test_index]

    ## Fit model
    SVC_pipe.fit(X_train_train, y_train_train)

    ## Use model to generate predictions on the holdout set
    pred = SVC_pipe.predict(X_holdout)

print(accuracy_score(y_holdout, 
               pred))

scores = cross_val_score(
    SVC_pipe, X_holdout, y_holdout,
    cv=kfold,
    scoring=make_scorer(balanced_accuracy_score),
    n_jobs=-1
)

print(f"CV balanced accuracy (holdout): {scores.mean():.3f} ± {scores.std():.3f}")



0.46078431372549017
CV balanced accuracy (holdout): 0.355 ± 0.059


In [47]:
roc_auc_score(y_holdout, SVC_pipe.decision_function(X_holdout))


np.float64(0.585423197492163)

In [59]:
#Multi-class case
roc_auc_score(y_holdout, SVC_pipe.predict_proba(X_holdout), multi_class='ovr')

np.float64(0.5981419533032416)

Baseline and MRI

In [49]:
Baseline_MRI_features = PD_merged_all[["GENDER", "AGE", "EDUC_YRS","lh_MeanThickness","lh_WhiteSurfArea", "lhCerebralWhiteMatterVol","Left_Cerebellum_White_Matter", "lhSurfaceHoles"  ]]
pd.get_dummies(Baseline_MRI_features['GENDER'],dtype=int)
Baseline_MRI_features['Male'] = pd.get_dummies(Baseline_MRI_features['GENDER'],dtype=int)['Male']
X_Baseline_MRI = Baseline_features.drop(columns=['GENDER'])

C:\Users\dcsor\AppData\Local\Temp\ipykernel_30668\68412693.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Baseline_MRI_features['Male'] = pd.get_dummies(Baseline_MRI_features['GENDER'],dtype=int)['Male']


In [50]:
#Split data 
X_train, X_test , y_train , y_test = train_test_split(X_Baseline_MRI, Y_target,
                                            shuffle=True,
                                            random_state=42,
                                            test_size=.2,
                                            stratify=Y_target)

#Make validation set with 15% of the
## training data in the validation set
X_train_train, X_val, y_train_train, y_val = train_test_split(
    X_train, y_train, shuffle=True, random_state=42, test_size=0.15, stratify=y_train
)

kfold = StratifiedKFold(n_splits=5,
                           shuffle=True,
                           random_state=42)
kfold.get_n_splits(X_train,y_train)


SVC_pipe = Pipeline([('scale', StandardScaler()), ('SVC', SVC(kernel = 'rbf', C=10,probability= True))])

for i, (train_index, test_index) in enumerate(kfold.split(X_train, y_train)):
    ## get the kfold training data
    X_train_train = X_train.iloc[train_index, :]
    y_train_train = y_train.iloc[train_index]

    ## get the holdout data
    X_holdout = X_train.iloc[test_index, :]
    y_holdout = y_train.iloc[test_index]

    ## Fit model
    SVC_pipe.fit(X_train_train, y_train_train)

    ## Use model to generate predictions on the holdout set
    pred = SVC_pipe.predict(X_holdout)

print(accuracy_score(y_holdout, 
               pred))

scores = cross_val_score(
    SVC_pipe, X_holdout, y_holdout,
    cv=kfold,
    scoring=make_scorer(balanced_accuracy_score),
    n_jobs=-1
)

print(f"CV balanced accuracy (holdout): {scores.mean():.3f} ± {scores.std():.3f}")

0.5784313725490197
CV balanced accuracy (holdout): 0.468 ± 0.114


In [51]:
roc_auc_score(y_holdout, SVC_pipe.decision_function(X_holdout))

np.float64(0.585423197492163)

Baseline and MRI and Genetic

In [52]:
PD_merged_allfeatures = PD_merged_all.drop(columns=['PATNO','MRIRSLT','rh_MeanThickness','rh_WhiteSurfArea', 'rhCerebralWhiteMatterVol','Right_Cerebellum_White_Matter', 'rhSurfaceHoles','CerebralWhiteMatterVol', 'NHY_BL', 'EVENT_ID','NHY'])
pd.get_dummies(PD_merged_allfeatures['GENDER'],dtype=int)
PD_merged_allfeatures['Male'] = pd.get_dummies(PD_merged_allfeatures['GENDER'],dtype=int)['Male']
X_features = PD_merged_allfeatures.drop(columns=['GENDER'])

In [53]:
#Split data 
X_train, X_test , y_train , y_test = train_test_split(X_features, Y_target,
                                            shuffle=True,
                                            random_state=42,
                                            test_size=.2,
                                            stratify=Y_target)

#Make validation set with 15% of the
## training data in the validation set
X_train_train, X_val, y_train_train, y_val = train_test_split(
    X_train, y_train, shuffle=True, random_state=42, test_size=0.15, stratify=y_train
)

kfold = StratifiedKFold(n_splits=5,
                           shuffle=True,
                           random_state=42)
kfold.get_n_splits(X_train,y_train)


SVC_pipe = Pipeline([('scale', StandardScaler()), ('SVC', SVC(kernel = 'rbf', C=10, probability=True))])

for i, (train_index, test_index) in enumerate(kfold.split(X_train, y_train)):
    ## get the kfold training data
    X_train_train = X_train.iloc[train_index, :]
    y_train_train = y_train.iloc[train_index]

    ## get the holdout data
    X_holdout = X_train.iloc[test_index, :]
    y_holdout = y_train.iloc[test_index]

    ## Fit model
    SVC_pipe.fit(X_train_train, y_train_train)

    ## Use model to generate predictions on the holdout set
    pred = SVC_pipe.predict(X_holdout)

print(accuracy_score(y_holdout, 
               pred))


scores = cross_val_score(
    SVC_pipe, X_holdout, y_holdout,
    cv=kfold,
    scoring=make_scorer(balanced_accuracy_score),
    n_jobs=-1
)

print(f"CV balanced accuracy (holdout): {scores.mean():.3f} ± {scores.std():.3f}")

0.696078431372549
CV balanced accuracy (holdout): 0.490 ± 0.042


In [54]:
roc_auc_score(y_holdout, SVC_pipe.decision_function(X_holdout))

np.float64(0.6974921630094043)